<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/OLID_Classification_GloVe_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNLSTM,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D,Input,Conv1D,MaxPooling1D,Flatten
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/dataset/OLID

/content/drive/My Drive/dataset/OLID


In [4]:
ls

elmo-model.h5      labels-levelb.csv       testset-levelb.tsv
glove.6B.100d.txt  labels-levelc.csv       testset-levelc.tsv
glove.6B.200d.txt  olid-training-v1.0.tsv
labels-levela.csv  testset-levela.tsv


In [5]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
train=pd.read_csv( 'olid-training-v1.0.tsv',sep="\t")
test=pd.read_csv('testset-levela.tsv',sep="\t")
y_test=pd.read_csv( 'labels-levela.csv',header=None).iloc[:,-1]
#OFF=0 
#NOT=1

In [7]:
y_train=train['subtask_a']
train=train['tweet']
test=test['tweet']
y_train=pd.factorize(y_train)[0]
y_test=pd.factorize(y_test)[0]

import collections
collections.Counter(y_train)

#Counter({0: 4400, 1: 8840})

Counter({0: 4400, 1: 8840})

In [0]:
# Creating the training corpus
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
corpus_train = []
for i in train:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_train.append(x)    
# Creating the training corpus
corpus_test = []
for i in test:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_test.append(x) 

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 1000 #frequency of words to be kept
max_len = 200 #MAX_SEQUENCE_LENGTH 

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(corpus_train)
sequences = tokenize.texts_to_sequences(corpus_train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [10]:
import os
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

embedding_dim = 100 #EMBEDDING_DIM 

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

1001


In [0]:
embedding_layer = Embedding(num_words,
                            embedding_dim,weights=[embedding_matrix],
                            input_length=max_len,trainable=False)

In [13]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(3)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(1, activation='sigmoid')(l_dense)

model = Model(sequence_input, preds)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))
model.fit(sequences_matrix,y_train,batch_size=256,epochs=5,verbose=2,class_weight=class_weights)

Epoch 1/5
 - 2s - loss: 0.6552 - acc: 0.6434
Epoch 2/5
 - 1s - loss: 0.6031 - acc: 0.7103
Epoch 3/5
 - 0s - loss: 0.5872 - acc: 0.7208
Epoch 4/5
 - 0s - loss: 0.5598 - acc: 0.7423
Epoch 5/5
 - 0s - loss: 0.5350 - acc: 0.7544


In [14]:
test_sequences = tokenize.texts_to_sequences(corpus_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
print(model.evaluate(test_sequences_matrix,y_test))

860/860 [==============================] - 0s 170us/step
[0.5404642222925674, 0.760465115724608]


In [15]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_sequences_matrix, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred))


860/860 [==============================] - 0s 119us/step
              precision    recall  f1-score   support

           0       0.58      0.54      0.56       240
           1       0.83      0.85      0.84       620

    accuracy                           0.76       860
   macro avg       0.70      0.69      0.70       860
weighted avg       0.76      0.76      0.76       860



In [0]:
import keras
keras.backend.clear_session()